In [2]:
%matplotlib inline

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from ggplot import *

from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.metrics import accuracy_score

import sys
sys.path.append('/Users/wooyoung/workarea/Kaggle/Shelter-Animal-Outcomes/kaggle/libs/')

import data_cleaning


In [3]:
df = pd.read_csv("data/train.csv")

In [4]:
df = data_cleaning.classify_colors(df)
df = data_cleaning.massage_df(df)

In [6]:
df_dog = df[df.AnimalType=='Dog']
df_cat = df[df.AnimalType=='Cat']


df_cat['catbreed'] = df_cat.Breed.apply(data_cleaning.fixCatBreed)

/Users/wooyoung/miniconda2/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


# CATZZZ

In [7]:
cat_features = ['age_numeric', 'neuter_status', 'sex', 'mixed', 'catbreed']
cat_features_dummies = ['age_numeric', 'neuter_status_Intact', 'neuter_status_Neutered',
       'neuter_status_Spayed', 'neuter_status_Unknown', 'sex_Female',
       'sex_Male', 'sex_Unknown', 'mixed_Mixed',
       'catbreed_domestic longhair', 'catbreed_domestic mediumhair',
       'catbreed_domestic shorthair', 'catbreed_rare', 'catbreed_siamese']
colors = ['has_Blue', 'has_Tortie',
       'has_White', 'has_Brown', 'has_Lilac', 'has_Point', 'has_Tabby',
       'has_Gray', 'has_Agouti', 'has_Black', 'has_Lynx', 'has_Orange',
       'has_Tan', 'has_Calico']

In [8]:
df_cat_dummies = pd.get_dummies(df_cat[cat_features + colors].dropna())

In [9]:
rows = np.random.choice(df_cat_dummies.index.values, len(df_cat_dummies)*.666, replace=False)
df_cat_dummies_train = df_cat_dummies.ix[rows]
df_cat_dummies_test = df_cat_dummies.drop(rows)

/Users/wooyoung/miniconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  if __name__ == '__main__':


### Multinomial GB

In [10]:
mnb = MultinomialNB()
mnb.fit(df_cat_dummies_train[cat_features_dummies + colors], df_cat.ix[df_cat_dummies_train.index.values]['OutcomeType'])

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [11]:
predictions = mnb.predict(df_cat_dummies_test[cat_features_dummies + colors])
accuracy_score(df_cat.ix[df_cat_dummies_test.index.values]['OutcomeType'], predictions)

0.71230598669623058

In [14]:
predictions_train = mnb.predict(df_cat_dummies_train[cat_features_dummies + colors])
accuracy_score(df_cat.ix[df_cat_dummies_train.index.values]['OutcomeType'], predictions_train)

0.71319338245516473

In [13]:
print 'classes:', mnb.classes_
print '\n', mnb.coef_

classes: ['Adoption' 'Died' 'Euthanasia' 'Return_to_owner' 'Transfer']

[[ -1.77754091  -4.86381287  -2.70655002  -2.71473764  -9.91366888
   -2.65244379  -2.65666617  -9.91366888  -1.9617575   -4.88978836
   -4.43302996  -2.20795606  -5.24084005  -5.08535514  -3.92220433
   -4.74888291  -2.98415211  -3.23658542  -7.27461155  -4.76037729
   -2.71998306  -5.98184325  -9.2205217   -3.36401814  -5.87061761
   -3.90240171  -5.00101399  -4.87671628]
 [ -1.93288633  -2.29920127  -4.49642585  -4.1397509   -4.1397509
   -2.85881706  -2.68113588  -4.1397509   -1.97642788  -4.24511142
   -4.65057653  -2.23764338  -5.05604164  -4.83289808  -3.95742935
   -5.34372371  -2.94582844  -3.26428217  -6.442336    -4.49642585
   -2.68113588  -5.05604164  -6.442336    -3.26428217  -5.74918882
   -3.80327867  -4.24511142  -5.34372371]
 [ -0.99862053  -2.72779548  -4.13559604  -4.41845883  -4.34435086
   -3.17120552  -2.97153985  -4.34435086  -2.24720974  -4.7061409
   -4.84334202  -2.50040564  -6.29026101  

In [232]:
for j in range(len(mnb.classes_)):
    print mnb.classes_[j]
    for i in range(len(cat_features_dummies + colors)):
        print (cat_features_dummies + colors)[i], mnb.coef_[j][i]
    print '\n'


Adoption
age_numeric -1.78499331841
neuter_status_Intact -4.82049181463
neuter_status_Neutered -2.71456366974
neuter_status_Spayed -2.70934950523
neuter_status_Unknown -9.91424201543
sex_Female -2.64045569759
sex_Male -2.66644943367
sex_Unknown -9.91424201543
mixed_Mixed -1.96057223678
catbreed_domestic longhair -4.99426108961
catbreed_domestic mediumhair -4.36516593054
catbreed_domestic shorthair -2.20493368205
catbreed_rare -5.25080292132
catbreed_siamese -5.11845146984
has_Blue -3.89808485574
has_Tortie -4.76674753862
has_White -2.99455816559
has_Brown -3.22464274626
has_Lilac -7.83480047375
has_Point -4.76095042094
has_Tabby -2.70416238726
has_Gray -5.85379900489
has_Agouti -9.22109483487
has_Black -3.3804531775
has_Lynx -5.85379900489
has_Orange -3.94298017564
has_Tan -4.97259959282
has_Calico -4.83283765045


Died
age_numeric -2.13790805604
neuter_status_Intact -2.23944516602
neuter_status_Neutered -4.78889033694
neuter_status_Spayed -4.20110367204
neuter_status_Unknown -4.095743